In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import os
import random

# directory = "/Users/Srikar/Desktop/Velexi/spectra-ml/data/plots"
directory = os.environ['DATA_DIR']
directory = os.path.join(directory, "plots")
os.chdir(directory)

In [2]:
num_samples = len(os.listdir(os.getcwd()))
img = mpimg.imread(os.path.join(directory,os.listdir(os.getcwd())[0]))
spectrum_height = img.shape[0]
spectrum_width = img.shape[1]

In [3]:
def convertimg(img):
    newimg = np.empty([img.shape[0], img.shape[1]])
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            row = img[i][j]
            newimg[i][j] = (row[0] + row[1] + row[2])/3
    return newimg

In [4]:
spectra = np.zeros((num_samples, spectrum_height, spectrum_width))
y = []
i = 0
for name in os.listdir():
    if name.find("Actinolite") != -1:
        y.append(int(0))
    elif name.find("Alunite") != -1:
        y.append(int(1))
    else:
        y.append(int(2))
    img = mpimg.imread(os.path.join(directory,name)) # os.path.join here, look into timeit, pickle file
    spectra[i] = convertimg(img)
    i += 1

In [5]:
spectra = spectra.reshape(spectra.shape[0], spectra.shape[1]*spectra.shape[2])

In [6]:
spectra.shape

(76, 124416)

In [7]:
y = np.reshape(y, (len(y), 1))
y.shape

(76, 1)

In [8]:
random.seed(0)

In [9]:
sample_indices = list(range(0, num_samples))
print(num_samples)
random.shuffle(sample_indices)
train_set_size = 3*(num_samples//5)
dev_set_size = (num_samples//5)
test_set_size= num_samples-dev_set_size - train_set_size
print(train_set_size)
print(test_set_size)
print(dev_set_size)
train_set_indices = sample_indices[:train_set_size]
dev_set_indices = sample_indices[train_set_size: train_set_size+dev_set_size]
test_set_indices= sample_indices[train_set_size+dev_set_size: num_samples]
print(train_set_indices)
print(test_set_indices)
print(dev_set_indices)

train_set = spectra[train_set_indices, :]
train_labels = y[train_set_indices, :]
dev_set = spectra[dev_set_indices, :]
dev_labels = y[dev_set_indices, :]
test_set = spectra[test_set_indices, :]
test_labels = y[test_set_indices, :]

76
45
16
15
[73, 52, 29, 11, 47, 63, 37, 1, 70, 67, 55, 40, 32, 74, 14, 42, 36, 24, 7, 10, 15, 50, 31, 58, 54, 25, 41, 12, 26, 23, 2, 0, 3, 56, 72, 28, 44, 60, 13, 20, 71, 22, 75, 35, 30]
[48, 8, 18, 17, 64, 27, 45, 61, 38, 51, 62, 65, 33, 5, 53, 49]
[21, 43, 4, 46, 59, 19, 9, 68, 57, 66, 34, 16, 69, 39, 6]


In [10]:
train_labels = train_labels.flatten()
dev_labels = dev_labels.flatten()
test_labels = test_labels.flatten()

In [11]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(bootstrap=True, criterion='entropy')

In [12]:
from sklearn.model_selection import train_test_split

y = np.reshape(y, (len(y), ))
X_train, X_test, y_train, y_test = train_test_split(spectra, y, test_size=0.2, random_state=42)
# clf.fit(train_set, train_labels)
clf.fit(X_train, y_train)

/Users/Srikar/Desktop/Programs/Virtual Environments/spectra-ml-env/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
from sklearn.metrics import accuracy_score
# preds = clf.predict(test_set)
# print("Accuracy:", accuracy_score(test_labels, preds))
preds = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))

Accuracy: 0.6875
